<a href="https://colab.research.google.com/github/Mlad-en/COV-BG/blob/main/Spearman_Pearson%20Correlations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openpyxl --upgrade

In [2]:
import pandas as pd
import openpyxl as xl
from scipy import stats

In [3]:
mortality_link = "https://github.com/Mlad-en/COV-BG/blob/main/data_output/Excess%20Mortality/Predicted/Countries/2020/Predicted_Excess_Mortality_EU_2020_%5B'Female',%20'Male',%20'Total'%5D_Total.xlsx?raw=true"
asyr_link = "https://github.com/Mlad-en/COV-BG/blob/main/data_output/PYLL/EU_ASYR_(0-4)-(85-89)_%5B'Female',%20'Male',%20'Total'%5D.xlsx?raw=true"
pyll_link = "https://github.com/Mlad-en/COV-BG/blob/main/data_output/PYLL/EU_PYLL_(0-4)-(85-89)_%5B'Female',%20'Male',%20'Total'%5D.xlsx?raw=true"
wyll_link = "https://github.com/Mlad-en/COV-BG/blob/main/data_output/PYLL/EU_WYLL_(15-19)-(60-64)_%5B'Female',%20'Male',%20'Total'%5D.xlsx?raw=true"
cvd_link = "https://raw.githubusercontent.com/Mlad-en/COV-BG/main/data_source/Population/Cardiovascular_diseases_Health_update_2021_cleaned.csv"

In [4]:
mort = pd.read_excel(mortality_link, sheet_name='Total', engine='openpyxl')
asyr = pd.read_excel(asyr_link, engine='openpyxl')
pyll = pd.read_excel(pyll_link, engine='openpyxl')
wyll = pd.read_excel(wyll_link, engine='openpyxl')
cvds = pd.read_csv(cvd_link)

In [5]:
asyr = asyr.loc[(asyr['Sex']=='Total'), ['Location', 'ASYR']]
pyll = pyll.loc[(pyll['Sex']=='Total'), ['Location', 'PYLL_STD_MEAN']]
wyll = wyll.loc[(wyll['Sex']=='Total'), ['Location', 'WYLL_STD_MEAN']]

asyr_pyll_wyll = asyr.merge(pyll, on='Location')
asyr_pyll_wyll = asyr_pyll_wyll.merge(wyll, on='Location')

In [6]:
mort = mort.loc[(mort['SIGNIFICANT']=='Significant Increase'), ['Location', 'Excess_mortality', 'P_Score', 'Excess_mortality_per_100_000']]

In [7]:
cvd_mort = mort.merge(cvds, on='Location')

In [8]:
cvd_mort_per_100_000 = cvd_mort.loc[:,['Location', 
                                       'Excess_mortality_per_100_000', 
                                       'P_Score',
                                       'Per 100_000-Total',
                                       'Per 100_000-LT65',]].copy()

In [9]:
cvd_mort_per_100_000 = cvd_mort_per_100_000.merge(asyr_pyll_wyll, on='Location')

In [10]:
relations = [
      ['P_Score', 'Per 100_000-Total'],
      ['Excess_mortality_per_100_000', 'Per 100_000-Total'],
      ['PYLL_STD_MEAN', 'Per 100_000-Total'],
      ['ASYR', 'Per 100_000-Total'],
      ['WYLL_STD_MEAN', 'Per 100_000-LT65'],
      ] 
mapping = {}

# Results

In [16]:
for rel in relations:
      pearson_coef, p_value = stats.pearsonr(cvd_mort_per_100_000[rel[0]], cvd_mort_per_100_000[rel[1]])
      spearman_coef, p_value_spearman = stats.spearmanr(cvd_mort_per_100_000[rel[0]], cvd_mort_per_100_000[rel[1]])
      key = " ".join([rel[0], "~", rel[1]])
      mapping[key] = {
                      'pearson_coef': f'{pearson_coef:.5f}',
                      'p_value': f'{p_value:.5f}',
                      'spearman_coef': f'{spearman_coef:.5f}',
                      'p_value_spearman': f'{p_value_spearman:.5f}',
                    }
pd.DataFrame(mapping).T

,pearson_coef,p_value,spearman_coef,p_value_spearman
P_Score ~ Per 100_000-Total,0.14775,0.49085,0.05610,0.79458
Excess_mortality_per_100_000 ~ Per 100_000-Total,0.52584,0.00831,0.34522,0.09850
PYLL_STD_MEAN ~ Per 100_000-Total,0.60926,0.00158,0.43304,0.03454
ASYR ~ Per 100_000-Total,0.58387,0.00274,0.42000,0.04102
WYLL_STD_MEAN ~ Per 100_000-LT65,0.66024,0.00045,0.46522,0.02198
